## Questo notebook scarica il calendario economico da investing.com
## Le date sono settabili come riportato sotto



In [2]:
#mounts current gdrive in colab after providing success code
#INSERISCI IL CODICE DI SUCCESSO O NON FUNZIONERA'
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/INVEST-IO/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/INVEST-IO


In [3]:
#apply filter on dates, scarica sempre dallo startdate (da indicare sotto) a enddate
#se enddate = today alla viene scaricato fino ad oggi dinamico, altrimenti imposta enddate alla data desiderata
#!!! FORMATO DATE US !!!
startdate = '01/01/2020'
enddate = '02/01/2020'

#importa la data di oggi nel formato corretto
from datetime import date
import time
if enddate == 'today':
  today = date.today()
  today_us_format = today.strftime("%m/%d/%Y")
else:
  today_us_format = enddate

In [4]:
%%capture
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as progress_bar
import pandas
import json

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [5]:
#function per cliccare pulsanti su chrome .click() di selenium non funziona
def click_on_button_id(x):
  element = wd.find_element_by_id(x)
  wd.execute_script("arguments[0].click();", element)

In [6]:
#CALENDARIO ECONOMICO

url = 'https://www.investing.com/economic-calendar'

wd.set_window_size(1920,1080)
wd.maximize_window()
wd.get(url)

#clicca sui pulsanti per filtrare la data
click_on_button_id('datePickerToggleBtn')
click_on_button_id('startDate')
wd.find_element_by_id("startDate").clear()
wd.find_element_by_id("startDate").send_keys(startdate)
click_on_button_id('endDate')
wd.find_element_by_id("endDate").clear()
wd.find_element_by_id("endDate").send_keys(today_us_format)
click_on_button_id("applyBtn")

#wait for the apply button to be effective
time.sleep(7)


#scrolla fino alla fine della pagina, aspetta questo timeout per caricare i dati
SCROLL_PAUSE_TIME = 6

#scrollare main method
last_height = wd.execute_script("return document.body.scrollHeight")
while True:
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = wd.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [7]:
tr_tuple = tuple(wd.find_elements_by_xpath('//*[@id="economicCalendarData"]/tbody//tr'))
print (str(len(tr_tuple)) + ' tr items to loop into, ' + str(len(tr_tuple)*6) + ' td items to loop')

1291 tr items to loop into, 7746 td items to loop


In [8]:
#launch main scraping loop
output_list = []

for tr in progress_bar(tr_tuple):
  td_tuple = tuple(tr.find_elements_by_css_selector('td'))
  if len(td_tuple) > 5:
    output_list.append({'date': calendar, 'time' : td_tuple[0].text, 
                        'currency' : td_tuple[1].text, 
                        'importance' : td_tuple[2].get_attribute('title'), 
                        'event' : td_tuple[3].text,
                        'actual' : td_tuple[4].text, 
                        'forecast': td_tuple[5].text, 
                        'previous' : td_tuple[6].text})
  elif len(td_tuple) > 2:
    output_list.append({'date': calendar, 'time' : td_tuple[0].text, 
                        'currency' : 'N/A', 
                        'importance' : td_tuple[2].text, 
                        'event' : td_tuple[3].text,
                        'actual' : 'N/A',
                        'forecast': 'N/A', 
                        'previous' : 'N/A'})
  else:
    calendar = td_tuple[0].text

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [9]:
from pandas import DataFrame
df = DataFrame (output_list)
df.to_csv('sr_economic_calendar_from_' + startdate.replace('/','') +  '_to_' + today_us_format.replace('/', '') + '.csv')

In [11]:
df.tail(15)

,date,time,currency,importance,event,actual,forecast,previous
1247,"Friday, January 31, 2020",15:30,USD,Low Volatility Expected,CFTC Silver speculative net positions,63.4K,,68.9K
1248,"Friday, January 31, 2020",15:30,USD,Low Volatility Expected,CFTC Soybeans speculative net positions,-6.3K,,17.6K
1249,"Friday, January 31, 2020",15:30,USD,Low Volatility Expected,CFTC Wheat speculative net positions,41.1K,,47.7K
1250,"Friday, January 31, 2020",15:30,CAD,Low Volatility Expected,CFTC CAD speculative net positions,34.6K,,38.3K
1251,"Friday, January 31, 2020",15:30,CHF,Low Volatility Expected,CFTC CHF speculative net positions,3.5K,,1.5K
1252,"Friday, January 31, 2020",15:30,AUD,Moderate Volatility Expected,CFTC AUD speculative net positions,-27.5K,,-19.3K
1253,"Friday, January 31, 2020",15:30,BRL,Moderate Volatility Expected,CFTC BRL speculative net positions,-23.6K,,-21.3K
1254,"Friday, January 31, 2020",15:30,JPY,Moderate Volatility Expected,CFTC JPY speculative net positions,-36.0K,,-44.7K
1255,"Friday, January 31, 2020",15:30,NZD,Low Volatility Expected,CFTC NZD speculative net positions,2.1K,,1.8K
1256,"Friday, January 31, 2020",15:30,RUB,Moderate Volatility Expected,CFTC RUB speculative positions,29.1K,,29.7K
